In [1]:
import pandas as pd
import math
import numpy as np

In [12]:
miami_weather = pd.read_csv('miami_weather_2013_2018.csv')

In [15]:
miami_weather.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [17]:
miami_weather.columns

Index(['CITY', 'DATE', 'apparentTemperatureHigh',
       'apparentTemperatureHighTime', 'apparentTemperatureLow',
       'apparentTemperatureLowTime', 'apparentTemperatureMax',
       'apparentTemperatureMaxTime', 'apparentTemperatureMin',
       'apparentTemperatureMinTime', 'cloudCover', 'dewPoint', 'humidity',
       'icon', 'moonPhase', 'ozone', 'precipIntensity', 'precipIntensityMax',
       'precipIntensityMaxTime', 'precipProbability', 'precipType', 'pressure',
       'summary', 'sunriseTime', 'sunsetTime', 'temperatureHigh',
       'temperatureHighTime', 'temperatureLow', 'temperatureLowTime',
       'temperatureMax', 'temperatureMaxTime', 'temperatureMin',
       'temperatureMinTime', 'time', 'uvIndex', 'uvIndexTime', 'visibility',
       'windBearing', 'windGust', 'windGustTime', 'windSpeed'],
      dtype='object')

In [19]:
miami_weather_subset = miami_weather[['CITY', 'DATE', 'cloudCover', 'dewPoint', 
                                      'humidity', 'precipIntensity', 'precipProbability', 
                                      'precipType', 'pressure', 'summary', 'temperatureHigh', 
                                      'temperatureLow', 'uvIndex', 'visibility', 'windBearing', 
                                      'windGust', 'windSpeed']]

In [31]:
miami_weather_subset.DATE.replace(list(miami_weather_subset['DATE']), 
                                  [x.replace('T23:59:59', '') for x in miami_weather_subset['DATE']], 
                                  inplace = True)
miami_weather_subset.CITY.replace(['MIAMI'], ['Miami, FL'], inplace = True)

In [34]:
miami_07weather = miami_weather_subset[miami_weather_subset['DATE'].str.contains("-07-")]

In [36]:
miami_12weather = miami_weather_subset[miami_weather_subset['DATE'].str.contains("-12-")]